# TASK 1#

In [61]:
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('///tr')

#Check the length of the rows
print('Quantity of columns:',[len(T) for T in tr_elements[:1]])

Quantity of columns: [3]


In [62]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postal Code

2 Borough

3 Neighborhood



In [63]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [64]:
print('Number of rows',[len(C) for (title,C) in col])

Number of rows [181, 181, 181]


In [65]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [66]:
##Remove N/A Borough rows
df.rename(columns={'Postal Code\n':'Postal Code'}, inplace=True)
df.rename(columns={'Borough\n':'Borough'}, inplace=True)
df.rename(columns={'Neighborhood\n':'Neighborhood'}, inplace=True)
df['Borough'] = df['Borough'].str.replace('\n','')
df['Neighborhood'] = df['Neighborhood'].str.replace('\n','')
df['Postal Code'] = df['Postal Code'].str.replace('\n','')

df.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [67]:
import numpy as np

# replace "Not assigned" to NaN
df.replace("Not assigned", np.nan, inplace = True)
df.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [68]:

missing_data = df.isnull()
missing_data.head(5)

,Postal Code,Borough,Neighborhood
0,False,True,True
1,False,True,True
2,False,False,False
3,False,False,False
4,False,False,False


In [69]:
##ignore Nan Borough
df=df.dropna(subset=['Borough'], axis=0)
df.head(12)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [70]:
##re indexing
df1=df.reset_index()
df2=df1[['Borough', 'Postal Code', 'Neighborhood']]
df2.head(12)

,Borough,Postal Code,Neighborhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,"Regent Park, Harbourfront"
3,North York,M6A,"Lawrence Manor, Lawrence Heights"
4,Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government"
5,Etobicoke,M9A,"Islington Avenue, Humber Valley Village"
6,Scarborough,M1B,"Malvern, Rouge"
7,North York,M3B,Don Mills
8,East York,M4B,"Parkview Hill, Woodbine Gardens"
9,Downtown Toronto,M5B,"Garden District, Ryerson"


In [71]:
print('Borough Data Frame Shape', df2.shape)

Borough Data Frame Shape (104, 3)


# TASK 2##

In [72]:
##get postal code coordinates data
path='https://cocl.us/Geospatial_data'
df3 = pd.read_csv(path)
df3

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [73]:
##merge both data frames
right = df3
left = df2
result = pd.merge(left, right, on=['Postal Code'])
result

,Borough,Postal Code,Neighborhood,Latitude,Longitude
0,North York,M3A,Parkwoods,43.753259,-79.329656
1,North York,M4A,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,M6A,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,Etobicoke,M9A,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,Scarborough,M1B,"Malvern, Rouge",43.806686,-79.194353
7,North York,M3B,Don Mills,43.745906,-79.352188
8,East York,M4B,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937


# TASK 3#

In [74]:
CLIENT_ID = 'yQSP4L4U10YG1FLCCH0412GUIIV5XVARIGMLGZO50XM4SGPUF' # your Foursquare ID
CLIENT_SECRET = 'WF5ID5FEGK0AG25K0T0W1YZ4A4NDBVEIO1N1P2UZM2O4TARM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: yQSP4L4U10YG1FLCCH0412GUIIV5XVARIGMLGZO50XM4SGPUF
CLIENT_SECRET:WF5ID5FEGK0AG25K0T0W1YZ4A4NDBVEIO1N1P2UZM2O4TARM


In [75]:
##explore data
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(result['Borough'].unique()),
        result.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [76]:
##let´s see all 10 Boroughs
result['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [77]:
!conda install -c conda-forge folium=0.5.0 --yes

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


In [78]:
#filtering Toronto Boroughs
central_data = result[result['Borough'] == 'Central Toronto']
down_data = result[result['Borough'] == 'Downtown Toronto']
east_data = result[result['Borough'] == 'East Toronto']
west_data = result[result['Borough'] == 'West Toronto'] 
data1 = central_data.append(down_data, ignore_index=True)
data2 = data1.append(east_data, ignore_index=True)
toronto_data = data2.append(west_data, ignore_index=True)
toronto_data

,Borough,Postal Code,Neighborhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M5N,Roselawn,43.711695,-79.416936
2,Central Toronto,M4P,Davisville North,43.712751,-79.390197
3,Central Toronto,M5P,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,Central Toronto,M4R,"North Toronto West, Lawrence Park",43.715383,-79.405678
5,Central Toronto,M5R,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,Central Toronto,M4S,Davisville,43.704324,-79.388790
7,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160
8,Central Toronto,M4V,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
9,Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636


In [79]:
import folium

In [80]:
# create map of toronto 
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto